## Data source
1. [Lichess Elite Database](https://database.nikonoel.fr/): Games between top players (elo 2200+)
2. Lichess beginner games
3. [CCRL](https://computerchess.org.uk/ccrl/Chess324/games.html): Computer vs computer games (elo 3000+)

### Example from Source 1

In [1]:
import chess.pgn, chess.engine
from stockfish import Stockfish
import random, math

In [9]:
with open("data/lichess_elite_2025-11.pgn", 'r') as f:
    while True:
        game = chess.pgn.read_game(f)
        if game is None: break

        print(game)
        sampled_data = process_pgn(game)
        #print(sampled_data)
        break

[Event "Rated Blitz game"]
[Site "?"]
[Date "2025.11.01"]
[Round "-"]
[White "Harald_Blatand"]
[Black "Misterchess64"]
[Result "0-1"]
[LichessURL "https://lichess.org/HOLYoRWL"]
[WhiteElo "2518"]
[BlackElo "2571"]
[ECO "B02"]
[Opening "Alekhine Defense: Sämisch Attack"]
[TimeControl "180+0"]
[UTCDate "2025.11.01"]
[UTCTime "00:00:34"]
[Termination "Normal"]
[WhiteRatingDiff "-5"]
[BlackRatingDiff "+5"]

1. e4 Nf6 2. e5 Nd5 3. Nc3 Nxc3 4. dxc3 d5 5. Nf3 c5 6. Bf4 Nc6 7. c4 d4 8. Bd3 Bg4 9. h3 Bxf3 10. Qxf3 e6 11. Qg3 Qa5+ 12. Bd2 Qc7 13. f4 g6 14. O-O h5 15. Rae1 O-O-O 16. a3 h4 17. Qf3 Ne7 18. b4 Nf5 19. bxc5 Bxc5 20. Rb1 b6 21. a4 Ne3 22. Rfe1 a5 23. Rb5 Kd7 24. Kh1 Ke7 25. Qe4 Nf5 26. Qf3 Ne3 27. Qe4 Nf5 28. Qf3 Rc8 29. Reb1 Rb8 30. Bxa5 Ne3 31. Bb4 Rhc8 32. Bxc5+ bxc5 33. Rb7 Kf8 34. Be4 Rxb7 35. Rxb7 Qa5 36. f5 Qe1+ 37. Kh2 Nf1+ 38. Kg1 Ng3+ 0-1


In [10]:
print(sampled_data[0]["output"])


<Step1>
The move number is 35. The next player is White.
<Step2>
 - Candidate Bxb7: let's analyze this candidate, the win rate is 52.48%
 - Candidate Rxb7: let's analyze this candidate, the win rate is 50.0%
 - Candidate Ra1: let's analyze this candidate, the win rate is 22.11%
<Step3>
Considering the above evaluation, the move Bxb7 is the optimal choice
<Step4>
<answer>Next player: White
Next move: Bxb7
Win rate: 22.11%


## How to construct reasoning trace

1. get some candidate moves
    * top-k from Stockfish
    * some good moves, some bad moves from Lichess database
    * top-k of some categories of moves
2. for each move, the analysis info can fome from:
    * stockfish info
    * tablebases
    * endgame
    * commentary (instructional games for beginners: how to reacess your chess)
    * critical position classifier

* https://github.com/hebbarashwin/lichess-puzzler/tree/master/tagger
* 

### Get analysis from Stockfish 
1. top-k moves, for each candidate move:
    * root move, win rate
    * (Optional) principle variation line (`length = L`)

In [3]:
STOCKFISH_PATH = "/usr/local/bin/stockfish" 
sf = Stockfish(path=STOCKFISH_PATH)

In [4]:
sf.set_fen_position("rnb1k2r/ppp2p2/1b1p4/3Pp3/4P3/2PBBPPq/PP2Q3/RN3RK1 b q - 0 18")
mvs = sf.get_top_moves(3)
mvs

[{'Move': 'b6e3', 'Centipawn': None, 'Mate': 2},
 {'Move': 'h3g3', 'Centipawn': 806, 'Mate': None},
 {'Move': 'h8g8', 'Centipawn': 736, 'Mate': None}]

In [5]:
m = mvs[0]
pv = m.get("Line") or m.get("PV") or m.get("Pv") or None
print(pv)

None


In [20]:
board = chess.Board("rnb1k2r/ppp2p2/1b1p4/3Pp3/4P3/2PBBPPq/PP2Q3/RN3RK1 b q - 0 18")
top_k = 3
depth = 15
limit = chess.engine.Limit(depth=int(depth))
pv_max_len = 5

with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
    infos = engine.analyse(board, limit, multipv=int(top_k))
    for info in infos:
        rank = int(info["multipv"])
        pv_moves = info.get("pv", [])
        print(pv_moves)
        pv_uci = [m.uci() for m in pv_moves]
        pv_uci = pv_uci[:pv_max_len]
        root_move = pv_uci[0]
        item = {
            "rank": rank,
            "move": root_move,
            "pv": pv_uci,
        }
        print(item)

[Move.from_uci('b6e3'), Move.from_uci('f1f2'), Move.from_uci('h3h1')]
{'rank': 1, 'move': 'b6e3', 'pv': ['b6e3', 'f1f2', 'h3h1']}
[Move.from_uci('h3g3'), Move.from_uci('e2g2'), Move.from_uci('b6e3'), Move.from_uci('f1f2'), Move.from_uci('e3f2'), Move.from_uci('g1f1'), Move.from_uci('g3g2'), Move.from_uci('f1g2'), Move.from_uci('f2c5'), Move.from_uci('g2f1'), Move.from_uci('c7c6'), Move.from_uci('d5c6'), Move.from_uci('b8c6'), Move.from_uci('f1e2'), Move.from_uci('e8d8'), Move.from_uci('b1d2'), Move.from_uci('h8h2'), Move.from_uci('e2e1')]
{'rank': 2, 'move': 'h3g3', 'pv': ['h3g3', 'e2g2', 'b6e3', 'f1f2', 'e3f2']}
[Move.from_uci('h8g8'), Move.from_uci('d3b5'), Move.from_uci('c7c6'), Move.from_uci('e2g2'), Move.from_uci('b6e3'), Move.from_uci('f1f2'), Move.from_uci('e3f2'), Move.from_uci('g1f2'), Move.from_uci('h3g2'), Move.from_uci('f2g2'), Move.from_uci('c6b5'), Move.from_uci('b1a3')]
{'rank': 3, 'move': 'h8g8', 'pv': ['h8g8', 'd3b5', 'c7c6', 'e2g2', 'b6e3']}


In [16]:
infos

[{'string': 'NNUE evaluation using nn-37f18f62d772.nnue (6MiB, (22528, 128, 15, 32, 1))',
  'depth': 15,
  'seldepth': 4,
  'multipv': 1,
  'score': PovScore(Mate(+2), BLACK),
  'nodes': 21444,
  'nps': 824769,
  'hashfull': 8,
  'tbhits': 0,
  'time': 0.026,
  'pv': [Move.from_uci('b6e3'), Move.from_uci('f1f2'), Move.from_uci('h3h1')]},
 {'depth': 15,
  'seldepth': 24,
  'multipv': 2,
  'score': PovScore(Cp(+806), BLACK),
  'nodes': 21444,
  'nps': 824769,
  'hashfull': 8,
  'tbhits': 0,
  'time': 0.026,
  'pv': [Move.from_uci('h3g3'),
   Move.from_uci('e2g2'),
   Move.from_uci('b6e3'),
   Move.from_uci('f1f2'),
   Move.from_uci('e3f2'),
   Move.from_uci('g1f1'),
   Move.from_uci('g3g2'),
   Move.from_uci('f1g2'),
   Move.from_uci('f2c5'),
   Move.from_uci('g2f1'),
   Move.from_uci('c7c6'),
   Move.from_uci('d5c6'),
   Move.from_uci('b8c6'),
   Move.from_uci('f1e2'),
   Move.from_uci('e8d8'),
   Move.from_uci('b1d2'),
   Move.from_uci('h8h2'),
   Move.from_uci('e2e1')]},
 {'depth': 15

In [6]:
def get_win_rate(cp, mate, turn_color):
    # Reference: https://lichess.org/page/accuracy
    # Win% = 50 + 50 * (2 / (1 + exp(-0.00368208 * centipawns)) - 1)
    if mate is not None:
        if mate > 0: return 100.0  
        if mate < 0: return 0.0    
        return 50.0 

    if cp is None: 
        return 50.0

    clamped_cp = max(min(cp, 4000), -4000)
    expiration = math.exp(-0.00368208 * clamped_cp)
    sigmoid = 1 / (1 + expiration)
    
    win_percent = 100 * sigmoid
    
    return round(win_percent, 2)

def analyze_position(board, top_k=10, depth=15):
    fen = board.fen()
    sf.set_depth(depth)
    sf.set_fen_position(fen)

    top_moves = sf.get_top_moves(top_k)

    candidates = []
    best_move_san = None
    best_win_rate = 50

    for i, move_data in enumerate(top_moves):
        uci = move_data['Move']
        cp = move_data.get('Centipawn')
        mate = move_data.get('Mate')

        san = board.san(chess.Move.from_uci(uci))

        win_rate = get_win_rate(cp, mate, board.turn)

        stockfish_info = {
            "rank": i + 1,
            "uci": uci,
            "san": san,
            "cp": cp,
            "mate": mate,
            "win_rate": win_rate
        }
        candidates.append(stockfish_info)
        if i == 0:
            best_move_san = san
            best_win_rate = win_rate
    
    return {
        "fen": fen,
        "turn_color": "White" if board.turn == chess.WHITE else "Black",
        "move_number": board.fullmove_number,
        "candidates": candidates,
        "best_move": best_move_san,
        "best_win_rate": best_win_rate
    }
        

### Reasoning template

In [7]:
def format_reasoning_trace(stockfish_info):
    
    turn = stockfish_info['turn_color']
    move_num = stockfish_info['move_number']
    best_move = stockfish_info['best_move']
    win_rate = stockfish_info['best_win_rate']

    steps = {}
    steps[1] = f"The move number is {move_num}. The next player is {turn}."
    
    cand_lines = []
    for cand in stockfish_info['candidates']:
        san = cand['san']
        win_rate = cand['win_rate']
        cand_lines.append(f" - Candidate {san}: let's analyze this candidate, the win rate is {win_rate}%")
    steps[2] = "\n".join(cand_lines)

    steps[3] = f"Considering the above evaluation, the move {best_move} is the optimal choice"

    steps[4] = f"<answer>Next player: {turn}\nNext move: {best_move}\nWin rate: {win_rate}%"

    reasoning = ""
    for i in range(1, 5):
        reasoning += f"\n<Step{i}>\n"
        reasoning += steps[i]
    return reasoning
    

In [8]:
def process_pgn(game, n_samples=1):
    board = game.board()
    positions = []
    entries = []
    for move in game.mainline_moves():
        positions.append(board.copy())
        board.push(move)

        if len(positions) > 10:
            indices = sorted(random.sample(range(5, len(positions) - 5), min(n_samples, len(positions) - 10)))
        else:
            indices = []

    for idx in indices:
        board_state = positions[idx]
        stockfish_info = analyze_position(board_state, top_k=3)
        reasoning_trace = format_reasoning_trace(stockfish_info)

        entry = {
            "system_prompt": "You are a chess Grandmaster. ... task description + format description",
            "user_input": f"FEN: {stockfish_info['fen']}",
            "output": reasoning_trace,
            "stockfish_info": stockfish_info,
        }
        entries.append(entry)
    return entries
            

## Two types of data in Go-dataset

### First
```
<reasoning>
首先我需要仔细观察当前的棋局状态。黑棋刚刚在E10落子，这明显是针对白棋在D8的防守。从棋形上看，这是一个关键的攻防点。

左上角区域有一个激烈的战斗正在进行。白棋在D16、C14、C15等位置有子，黑棋则有D14、C13等子。白棋在这个区域形成了一定的优势，但黑棋通过D10和最新的E10落子正在积极反击。

现在轮到白棋第80手，我需要考虑几个关键选点：

选项一：白棋下在D12
这是一个直接的应对，目的是切断黑棋在左上区域的连接。D12可以与白棋此前在E12的子形成联系，构建一个强大的结构。如果白棋选择D12，可能的变化是：
80.O-D12
81.X-F12（黑棋可能会尝试保护自己的棋形）
82.O-E13（白棋可以继续施压）
83.X-F13（黑棋可能会尝试维持平衡）
84.O-E14（白棋形成一个强大的形状）
85.X-J2（黑棋可能会转向其他区域反击，比如照顾J2这个之前被忽略的点）
86.O-H2
87.X-K2

通过计算，这种变化下白棋的胜率约为71.2%，这是一个相当有利的优势。

选项二：白棋下在M2
这是一个更为间接的选择，可能是想在右下区域发展或者干扰黑棋在那里的计划。M2可以与白棋此前在N3的子形成联系，构建一个结构。如果选择这条路线，可能的变化是：
80.O-M2

通过计算，这种变化下白棋的胜率约为62.8%，明显不如D12的选择。

我还注意到，黑棋在E10的落子显然是想要在这个区域展开战斗。白棋之前在D12的落子似乎是为了预防或准备这种战斗。现在白棋在D12的落子可以进一步强化在这个区域的优势。

综合考虑棋形、战略和计算结果，D12明显是更优的选择。它不仅能够直接应对黑棋的最新动向，形成更有效的结构，同时也能够提供更高的胜率。
</reasoning>

<answer>
\boxed{下一步颜色:白}
\boxed{下一步位置:D12}
\boxed{下一步胜率:71.2%}
</answer>
```

```
<reasoning>
First, I need to carefully observe the current board position. Black has just played at E10, which is clearly aimed at White’s defense at D8. From the shape of the position, this is a critical point of attack and defense.

There is an intense fight going on in the upper-left corner. White has stones at D16, C14, C15, and other points, while Black has stones at D14, C13, and so on. White has formed a certain advantage in this area, but Black is actively counterattacking through D10 and the latest move at E10.

Now it is White’s turn on move 80, and I need to consider several key candidate points:

Option 1: White plays at D12  
This is a direct response, aimed at cutting Black’s connections in the upper-left area. D12 can connect with White’s earlier stone at E12, building a strong structure. If White chooses D12, a possible variation is:
80. O-D12  
81. X-F12 (Black may try to protect their shape)  
82. O-E13 (White can continue to apply pressure)  
83. X-F13 (Black may try to maintain balance)  
84. O-E14 (White forms a strong shape)  
85. X-J2 (Black may turn to counterattack elsewhere, such as taking care of J2, a point that was previously neglected)  
86. O-H2  
87. X-K2  

According to calculations, in this variation White’s winning probability is about 71.2%, which is a very favorable advantage.

Option 2: White plays at M2  
This is a more indirect choice, possibly aiming to develop in the lower-right area or interfere with Black’s plans there. M2 can connect with White’s earlier stone at N3, forming a structure. If this line is chosen, a possible continuation is:
80. O-M2  

According to calculations, in this variation White’s winning probability is about 62.8%, clearly inferior to the D12 choice.

I also notice that Black’s move at E10 is clearly intended to start a fight in this area. White’s earlier consideration of D12 seems to have been meant to prevent or prepare for this kind of battle. Now, playing D12 can further strengthen White’s advantage in this area.

Taking into account the board position, strategy, and calculation results, D12 is clearly the better choice. It not only directly responds to Black’s latest move and forms a more effective structure, but also offers a higher winning probability.
</reasoning>

<answer>
\boxed{Next move color: White}  
\boxed{Next move position: D12}  
\boxed{Next move winning rate: 71.2%}
</answer>
```

### Second
```
<Step1>
最后一步是59.X-S2，因此当前进行到第59步，下一步是第60步，轮到白棋落子。
<Step2>
让我们来为白棋分析接下来可能的走法：
首先考虑60.O-S3
下在这里的话，后续的变化可能是：
60.O-S3
61.X-S3
62.O-S3
63.X-S3
64.O-S3
65.X-S3
66.O-S3
67.X-S3
在这种变化下，白棋的胜率约为57.5%
另一个可能的选择是60.O-R9
这样下的话，后续可能变化为：
60.O-R9
61.X-R9
62.O-R9
63.X-R9
64.O-R9
65.X-R9
66.O-R9
67.X-R9
在以上的进行后，白棋的胜率为55.9%
接下来分析60.O-P2
后面可能会这样进行：
60.O-P2
61.X-P2
62.O-P2
63.X-P2
64.O-P2
考虑以上的变化，白棋的胜率是55.2%
等等，让我们再考虑一下60.O-M2
这样下的话，后续可能变化为：
60.O-M2
61.X-M2
62.O-M2
63.X-M2
64.O-M2
65.X-M2
66.O-M2
67.X-M2
在以上的进行后，白棋的胜率为57.1%
等等，让我们再考虑一下60.O-Q9
这样下的话，后续可能变化为：
60.O-Q9
61.X-Q9
62.O-Q9
63.X-Q9
考虑以上的变化，白棋的胜率是54.4%
再想想，可以尝试60.O-N2
如果选择这个走法，后续可能的变化是：
60.O-N2
61.X-N2
62.O-N2
63.X-N2
64.O-N2
65.X-N2
66.O-N2
67.X-N2
在以上的进行后，白棋的胜率为55.7%
另一个可能的选择是60.O-B14
后面可能会这样进行：
60.O-B14
61.X-B14
62.O-B14
63.X-B14
64.O-B14
65.X-B14
66.O-B14
67.X-B14
考虑以上的变化，白棋的胜率是55.4%
<Step3>
综合以上分析，最佳的走法是60.O-S3，
选择这个下法，后续可能的变化为：
60.O-S3
61.X-S3
62.O-S3
63.X-S3
64.O-S3
65.X-S3
66.O-S3
67.X-S3
这个选择能够带来57.5%的胜率。
<Step4>
<answer>
\boxed{下一步颜色:白}
\boxed{下一步位置:S3}

</answer>
```

```
<Step1>
The last move is 59.X-S2, so the game is currently at move 59. The next move is move 60, and it is White’s turn to play.
<Step2>
Let’s analyze White’s possible next moves:
First, consider 60.O-S3.
If played here, the possible continuation might be:
60.O-S3
61.X-S3
62.O-S3
63.X-S3
64.O-S3
65.X-S3
66.O-S3
67.X-S3
Under this line, White’s win rate is about 57.5%.
Another possible choice is 60.O-R9.
If played this way, the possible continuation is:
60.O-R9
61.X-R9
62.O-R9
63.X-R9
64.O-R9
65.X-R9
66.O-R9
67.X-R9
After this sequence, White’s win rate is 55.9%.
Next, analyze 60.O-P2.
It might proceed like this:
60.O-P2
61.X-P2
62.O-P2
63.X-P2
64.O-P2
Considering this line, White’s win rate is 55.2%.
Wait—let’s also consider 60.O-M2.
If played this way, the possible continuation might be:
60.O-M2
61.X-M2
62.O-M2
63.X-M2
64.O-M2
65.X-M2
66.O-M2
67.X-M2
After this sequence, White’s win rate is 57.1%.
Wait—let’s also consider 60.O-Q9.
If played this way, the possible continuation might be:
60.O-Q9
61.X-Q9
62.O-Q9
63.X-Q9
Considering this line, White’s win rate is 54.4%.
Thinking further, you can try 60.O-N2.
If this move is chosen, the possible continuation is:
60.O-N2
61.X-N2
62.O-N2
63.X-N2
64.O-N2
65.X-N2
66.O-N2
67.X-N2
After this sequence, White’s win rate is 55.7%.
Another possible choice is 60.O-B14.
It might proceed like this:
60.O-B14
61.X-B14
62.O-B14
63.X-B14
64.O-B14
65.X-B14
66.O-B14
67.X-B14
Considering this line, White’s win rate is 55.4%.
<Step3>
Based on the above analysis, the best move is 60.O-S3.
Choosing this move, the possible continuation is:
60.O-S3
61.X-S3
62.O-S3
63.X-S3
64.O-S3
65.X-S3
66.O-S3
67.X-S3
This choice yields a 57.5% win rate.
<Step4>
<answer>
\boxed{Next move color: White}
\boxed{Next move position: S3}

</answer>
```